In [1]:
from utils import Banco
from dotenv import load_dotenv
import os

load_dotenv()

banco = Banco(
    database=os.getenv('DATABASE'),
    host=os.getenv('HOST'),
    user=os.getenv('USER'),
    password=os.getenv('PASSWORD')
)

In [3]:
with open('query.sql', 'r') as file:
    query = file.read()

df = banco.get_table(query, ano=2024, id_modulo=[58], analito=['Triglicerídeos'])
df


,part,id_modulo,analito,envio_str,envio,item_ensaio,sistema,valor
0,2522,58,Triglicerídeos,Jan/2024,2024-01-01,BQ03,2,101.0
1,2524,58,Triglicerídeos,Jan/2024,2024-01-01,BQ01,1,192.0
2,2524,58,Triglicerídeos,Jan/2024,2024-01-01,BQ03,1,85.0
3,82,58,Triglicerídeos,Out/2023,2023-10-01,BQ03,1,251.0
4,2543,58,Triglicerídeos,Jan/2024,2024-01-01,BQ03,1,81.0
...,...,...,...,...,...,...,...,...
22847,16427,58,Triglicerídeos,Abr/2024,2024-04-01,BQ01,1,105.0
22848,16461,58,Triglicerídeos,Abr/2024,2024-04-01,BQ03,1,183.0
22849,16589,58,Triglicerídeos,Abr/2024,2024-04-01,BQ03,1,138.0
22850,16614,58,Triglicerídeos,Abr/2024,2024-04-01,BQ02,1,198.0


In [122]:
from tratamento import PreProcessamento

prepross = PreProcessamento(df)
df_tratado = prepross.get_data()
df_tratado

item,part,id_modulo,analito,sistema,A24R1I1,A24R1I2,A24R1I3,A24R2I1,A24R2I2,A24R2I3,A24R3I1,A24R3I2,A24R3I3,A24R4I1,A24R4I2,A24R4I3
0,15,58,Triglicerídeos,1,89.0,97.0,204.0,182.0,185.0,87.0,86.0,155.0,135.0,179.0,85.0,91.0
1,28,58,Triglicerídeos,1,99.0,107.0,228.0,199.0,202.0,88.0,103.0,169.0,152.0,194.0,87.0,102.0
2,28,58,Triglicerídeos,2,100.0,105.0,233.0,195.0,202.0,90.0,100.0,167.0,148.0,200.0,87.0,104.0
3,34,58,Triglicerídeos,1,97.0,105.0,237.0,196.0,195.0,82.0,102.0,169.0,154.0,196.0,85.0,103.0
4,36,58,Triglicerídeos,1,102.0,112.0,242.0,196.0,207.0,92.0,100.0,172.0,153.0,205.0,84.0,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,16452,58,Triglicerídeos,1,NaN,NaN,NaN,184.0,170.0,79.0,91.0,152.0,135.0,183.0,71.0,83.0
2329,16453,58,Triglicerídeos,1,NaN,NaN,NaN,289.0,201.0,90.0,105.0,196.0,170.0,191.0,90.0,140.0
2331,16455,58,Triglicerídeos,1,NaN,NaN,NaN,187.0,179.0,81.0,105.0,170.0,136.0,185.0,79.0,91.0
2332,16457,58,Triglicerídeos,1,NaN,NaN,NaN,175.0,197.0,80.0,120.0,180.0,172.0,175.0,77.0,97.0


In [123]:
import pandas as pd
def cola_identica(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df.drop('part', axis=1).duplicated(keep=False)]

    df = df.copy()
    df['part'] = df['part'].astype(str)

    agregar = [i for i in df.columns if i not in ['part','sistema']]
    return df.groupby(agregar, dropna=False)['part'].agg(lambda x: ' - '.join((x))).reset_index()

len(cola_identica(df_tratado))

12

In [136]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np
from collections import Counter

def get_best_eps(df: pd.DataFrame, max_lim: float) -> float:
    i = 0.0001
    while True:
        model = DBSCAN(eps=i, min_samples=2, n_jobs=-1)
        model = model.fit(df)
        max_len_cluster = max(Counter(model.labels_).values())

        if max_len_cluster <= max_lim:
            return i
        if i>=0.01:
            i += .05
            continue
        if i>2:
            return 1e-5
        i += 0.0001

def apply_model(df: pd.DataFrame, eps=0.06) -> pd.DataFrame:
    # Preenchendo vazios pela média da linha
    imputer = SimpleImputer(strategy="mean")
    df_model = imputer.fit_transform(df.T).T

    # Aplicando a normalização
    norm = StandardScaler()
    df_model = norm.fit_transform(df_model)
    # dados = df_model.flatten()
    # q1 = np.percentile(dados, 25)
    # q3 = np.percentile(dados, 75)

    # Aplicando o modelo DBSCAN
    model = DBSCAN(eps=eps, min_samples=2, n_jobs=-1, )
    model = model.fit(df_model)

    df['cluster'] = model.labels_
    return df.query('cluster != -1').sort_values('cluster')


teste = df_tratado.set_index(['part','id_modulo','analito','sistema'])

apply_model(teste)#.reset_index()#.value_counts('cluster'))

,,,item,A24R1I1,A24R1I2,A24R1I3,A24R2I1,A24R2I2,A24R2I3,A24R3I1,A24R3I2,A24R3I3,A24R4I1,A24R4I2,A24R4I3,cluster
part,id_modulo,analito,sistema,,,,,,,,,,,,,
235,58,Triglicerídeos,1,95.0,92.0,222.0,192.0,201.0,101.0,96.0,149.0,128.0,178.0,72.0,88.0,0
5756,58,Triglicerídeos,1,95.0,92.0,222.0,192.0,201.0,101.0,96.0,149.0,128.0,178.0,72.0,88.0,0
904,58,Triglicerídeos,1,108.0,109.0,251.0,196.0,189.0,81.0,97.0,163.0,142.0,213.0,64.0,86.0,1
553,58,Triglicerídeos,1,108.0,109.0,251.0,196.0,189.0,81.0,97.0,163.0,142.0,213.0,64.0,86.0,1
806,58,Triglicerídeos,1,96.0,103.0,232.0,200.0,203.0,85.0,110.0,174.0,158.0,206.0,94.0,111.0,2
591,58,Triglicerídeos,1,96.0,103.0,232.0,200.0,203.0,85.0,110.0,174.0,158.0,206.0,94.0,111.0,2
1881,58,Triglicerídeos,1,89.0,91.0,223.0,203.0,202.0,84.0,106.0,177.0,155.0,219.0,92.0,112.0,3
818,58,Triglicerídeos,1,89.0,91.0,223.0,203.0,202.0,84.0,106.0,177.0,155.0,219.0,92.0,112.0,3
1961,58,Triglicerídeos,1,110.0,117.0,250.0,198.0,203.0,81.0,101.0,158.0,136.0,193.0,75.0,95.0,4
